In [22]:
import torch
import torch.nn as nn

# 模拟Generator类中的defactorize方法
def defactorize_modified(fact_list, feat_shape, conv, conv_combined):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = -len(_shape)

        # 分别处理并应用标准卷积到每个特征
        processed_facts = []
        for _fact in fact_list:
            _processed = conv(_fact[_key])
            processed_facts.append(_processed)

        # 合并处理后的特征
        _cat_facts = torch.cat(processed_facts, dim=_cat_dim)

        # 再次应用标准卷积到合并后的特征
        _cat_facts = conv_combined(_cat_facts)

        feats[_key] = _cat_facts  # 存储处理后的特征
    return feats

def defactorize_original(fact_list, feat_shape):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = -len(_shape)
        _cat_facts = torch.cat([_fact[_key] for _fact in fact_list], dim=_cat_dim)

        feats[_key] = _cat_facts  # 存储处理后的特征
    return feats

# 模拟输入数据
feat_shape = {"last": (256, 16, 16)}
fact_list = [{"last": torch.rand(256, 256, 3, 3) for _ in range(2)}]  # 模拟两个fact_list项

# 创建卷积层
conv = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)
conv_combined = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)

# 调用两个版本的defactorize方法
output_feats_modified = defactorize_modified(fact_list, feat_shape, conv, conv_combined)
output_feats_original = defactorize_original(fact_list, feat_shape)

# 比较两个输出的形状
for key in output_feats_modified:
    shape_modified = output_feats_modified[key].shape
    shape_original = output_feats_original[key].shape
    print(f"Feature key: {key}, Modified shape: {shape_modified}, Original shape: {shape_original}")


Feature key: last, Modified shape: torch.Size([256, 256, 3, 3]), Original shape: torch.Size([256, 256, 3, 3])


In [31]:

# 模拟Generator类中的defactorize方法
def defactorize_modified(fact_list, feat_shape, conv, conv_combined):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = 1  # 在特征（通道）维度上合并

        processed_facts = []
        for _fact in fact_list:
            # 应用第一个卷积层
            _processed = conv(_fact[_key])
            processed_facts.append(_processed)

        # 合并处理后的特征
        _cat_facts = torch.cat(processed_facts, dim=_cat_dim)

        # 应用第二个卷积层
        _cat_facts = conv_combined(_cat_facts)

        # 调整特征尺寸以匹配原始形状
        # 注意：这可能需要根据您的具体应用进行调整
        _cat_facts = _cat_facts.view(*_shape)

        feats[_key] = _cat_facts  # 存储处理后的特征

    return feats

def defactorize_original(fact_list, feat_shape):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = -len(_shape)
        _cat_facts = torch.cat([_fact[_key] for _fact in fact_list], dim=_cat_dim)

        feats[_key] = _cat_facts  # 存储处理后的特征
    return feats

# 模拟输入数据
feat_shape = {"last": (256, 16, 16)}
fact_list = [{"last": torch.rand(2, 256, 16, 16) for _ in range(2)}]  # 模拟两个fact_list项

# 创建卷积层
conv = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)
conv_combined = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)

# 调用两个版本的defactorize方法
output_feats_modified = defactorize_modified(fact_list, feat_shape, conv, conv_combined)
output_feats_original = defactorize_original(fact_list, feat_shape)

# 比较两个输出的形状
for key in output_feats_modified:
    shape_modified = output_feats_modified[key].shape
    shape_original = output_feats_original[key].shape
    print(f"Feature key: {key}, Modified shape: {shape_modified}, Original shape: {shape_original}")

RuntimeError: shape '[256, 16, 16]' is invalid for input of size 131072

In [3]:
import torch
import torch.nn as nn

def defactorize_test(fact_list, conv):
    for _fact in fact_list:
        for _key, _val in _fact.items():
            # 打印原始特征的形状
            print(f"Original shape of {_key}: {_val.shape}")

            # 调整特征为4维张量
            _val_reshaped = _val.view(-1, *(_val.shape[-3:]))
            print(f"Reshaped shape for convolution: {_val_reshaped.shape}")

            # 应用卷积
            _processed = conv(_val_reshaped)
            print(f"Shape after convolution: {_processed.shape}")

# 模拟输入数据
fact_list = [{"last": torch.rand(2, 6, 256, 16, 16)}]  # 模拟一个fact_list项，形状为[2, 6, 256, 16, 16]

# 创建卷积层
conv = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)

# 运行测试函数
defactorize_test(fact_list, conv)


Original shape of last: torch.Size([2, 6, 256, 16, 16])
Reshaped shape for convolution: torch.Size([12, 256, 16, 16])
Shape after convolution: torch.Size([12, 256, 16, 16])


In [4]:
import torch
import torch.nn as nn

def defactorize_test(fact_list, conv):
    for _fact in fact_list:
        for _key, _val in _fact.items():
            # 打印原始特征的形状
            print(f"Original shape of {_key}: {_val.shape}")

            # 调整特征为4维张量
            batch_size, num_experts, C, H, W = _val.shape
            _val_reshaped = _val.view(-1, C, H, W)  # 将批次大小和专家数量合并为一个维度
            print(f"Reshaped shape for convolution: {_val_reshaped.shape}")

            # 应用卷积
            _processed = conv(_val_reshaped)
            print(f"Shape after convolution: {_processed.shape}")

            # 恢复原始批次大小和专家数量的维度
            _processed_restored = _processed.view(batch_size, num_experts, C, H, W)
            print(f"Restored shape after convolution: {_processed_restored.shape}")

# 模拟输入数据
fact_list = [{"last": torch.rand(2, 6, 256, 16, 16)}]  # 模拟一个fact_list项，形状为[2, 6, 256, 16, 16]

# 创建卷积层
conv = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)

# 运行测试函数
defactorize_test(fact_list, conv)



Original shape of last: torch.Size([2, 6, 256, 16, 16])
Reshaped shape for convolution: torch.Size([12, 256, 16, 16])
Shape after convolution: torch.Size([12, 256, 16, 16])
Restored shape after convolution: torch.Size([2, 6, 256, 16, 16])


In [26]:
fact_list = [{"last": torch.rand(256, 256, 3, 3) for _ in range(2)}] 
conv = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)
conv_combined = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)

def defactorize_modified(fact_list, feat_shape, conv, conv_combined):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = -len(_shape)

        # 分别处理并应用标准卷积到每个特征
        processed_facts = []
        for _fact in fact_list:
            _processed = conv(_fact[_key])
            processed_facts.append(_processed)

        # 合并处理后的特征
        _cat_facts = torch.cat(processed_facts, dim=_cat_dim)

        # 再次应用标准卷积到合并后的特征
        _cat_facts = conv_combined(_cat_facts)

        feats[_key] = _cat_facts  # 存储处理后的特征
    return feats

# 测试原始的defactorize函数
def defactorize_original(fact_list, feat_shape):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = -len(_shape)
        _cat_facts = torch.cat([_fact[_key] for _fact in fact_list], dim=_cat_dim)

        feats[_key] = _cat_facts  # 存储处理后的特征
    return feats

# 调用两个版本的defactorize方法
output_feats_modified = defactorize_modified(fact_list, feat_shape, conv, conv_combined)
output_feats_original = defactorize_original(fact_list, feat_shape)

# 比较两个输出的形状
for key in output_feats_modified:
    shape_modified = output_feats_modified[key].shape
    shape_original = output_feats_original[key].shape
    print(f"Feature key: {key}, Modified shape: {shape_modified}, Original shape: {shape_original}")

Feature key: last, Modified shape: torch.Size([256, 256, 3, 3]), Original shape: torch.Size([256, 256, 3, 3])


In [32]:
fact_list = [{"last": torch.rand(8, 6, 256, 16, 16) for _ in range(2)}] 
def defactorize_original(fact_list, feat_shape):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = -len(_shape)
        _cat_facts = torch.cat([_fact[_key] for _fact in fact_list], dim=_cat_dim)

        feats[_key] = _cat_facts  # 存储处理后的特征
    return feats

# 定义卷积层
conv = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)
conv_combined = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=1, padding=1)
def defactorize_modified(fact_list, feat_shape, conv, conv_combined):
    feats = {}
    for _key in feat_shape:
        _shape = feat_shape[_key]
        _cat_dim = 1  # 在特征（通道）维度上合并

        processed_facts = []
        for _fact in fact_list:
            # 应用第一个卷积层
            _processed = conv(_fact[_key])
            processed_facts.append(_processed)

        # 合并处理后的特征
        _cat_facts = torch.cat(processed_facts, dim=_cat_dim)

        # 应用第二个卷积层
        _cat_facts = conv_combined(_cat_facts)

        # 调整特征尺寸以匹配原始形状
        # 注意：这可能需要根据您的具体应用进行调整
        _cat_facts = _cat_facts.view(*_shape)

        feats[_key] = _cat_facts  # 存储处理后的特征

    return feats

# 调用两个版本的defactorize方法
output_feats_modified = defactorize_modified(fact_list, feat_shape, conv, conv_combined)
output_feats_original = defactorize_original(fact_list, feat_shape)

# 比较两个输出的形状
for key in output_feats_modified:
    shape_modified = output_feats_modified[key].shape
    shape_original = output_feats_original[key].shape
    print(f"Feature key: {key}, Modified shape: {shape_modified}, Original shape: {shape_original}")

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [256, 256, 3, 3], but got 5-dimensional input of size [8, 6, 256, 16, 16] instead